In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import imp
from rtpt import RTPT
from tqdm.notebook import tqdm
import h5py
import gym
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.optim import Adam, SGD
from torch.nn import  CrossEntropyLoss, MSELoss

from stable_baselines3.common.env_checker import check_env
from stable_baselines3 import PPO

from main_config import main_config

from gym_env import RBCEnv
from nn.ppo_nn import CustomActorCriticPolicy
from train.utils import create_hdf5_generator

from gym.spaces import Space, Box

In [3]:
# H5_TRAIN = "..\\data\\top3-partial-flip-train.hdf5"
# H5_VAL = "..\\data\\top3-partial-flip-val.hdf5"
# H5_TEST = "..\\data\\top3-partial-flip-test.hdf5"

H5_TRAIN = main_config["train_dir"]
H5_VAL = main_config["val_dir"]
H5_TEST = main_config["test_dir"]

In [4]:
batch_size = 32

file = h5py.File(H5_TRAIN)
train_data_size = file['data'].shape[0]
file = h5py.File(H5_VAL)
val_data_size = file['data'].shape[0]
train_hdf5_generator = create_hdf5_generator(H5_TRAIN, batch_size)
val_hdf5_generator = create_hdf5_generator(H5_VAL, batch_size)

In [5]:
def fit(epochs, lr, model, train_loader, val_loader, criterion, opt_func = SGD):
    rtpt = RTPT(name_initials='AM', experiment_name='sense-pre-training', max_iterations=epochs)
    rtpt.start()
    best_val_loss = np.inf
    history = []
    optimizer = opt_func(model.parameters(), lr, weight_decay=0.01)
    for epoch in range(epochs):

        model.train()
        train_losses = []
        with tqdm(train_loader,  unit='batch', total = train_data_size // batch_size) as tepoch:
            for data, labels in tepoch: 
                # print('data shape ', data.shape)
                # print('labels shape ', labels.shape)
                tepoch.set_description(f"Epoch {epoch}")
                if labels.shape[0] < batch_size or data.shape[0] < batch_size: 
                    break
                # labels = labels.to(torch.float).unsqueeze(-1) # MSE
                labels = labels.to(torch.long) # CrossEntropyLoss
                # converting the data into GPU format
                if torch.cuda.is_available():
                    data, labels = data.cuda(), labels.cuda()

                loss = model.training_step(criterion, data, labels)
                train_losses.append(loss)
                loss.backward()
                optimizer.step()
                optimizer.zero_grad()

        model.eval()
        outputs = []
        with tqdm(val_loader, unit='batch', total = val_data_size // batch_size) as vepoch:
            for data, labels in vepoch:
                if labels.shape[0] < batch_size or data.shape[0] < batch_size: 
                    break
                # labels = labels.to(torch.float).unsqueeze(-1) # MSE
                # labels = labels.to(torch.long) # CrossEntropyLoss
                # converting the data into GPU format
                if torch.cuda.is_available():
                    data, labels = data.cuda(), labels.cuda()
                outputs.append(model.validation_step(criterion, data, labels))
        result = model.validation_epoch_end(outputs)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        model.epoch_end(epoch, result)
        history.append(result)
        
        if result['val_loss'] < best_val_loss: 
            val_loss = result['val_loss']
            print(f'Validation Loss Decreased({best_val_loss}--->{val_loss}) \t Saving The Model')
            best_val_loss = val_loss
            torch.save(model, '/root/rbc/models/pretrain_model.pth')

        rtpt.step()
        
    return history

In [ ]:
# if __name__ == '__main__':
env = RBCEnv()
    # check_env(env=env)

model = PPO(CustomActorCriticPolicy, env, verbose = 1)
policy = model.policy

opt_func = Adam
lr = 0.001
criterion = CrossEntropyLoss()
# criterion = MSELoss()
# criterion = DiceLoss()
num_epochs = 20

# check if GPU is available
if torch.cuda.is_available():
    policy = policy.cuda()
    criterion = criterion.cuda()

history = fit(num_epochs, lr, policy, train_hdf5_generator, val_hdf5_generator, criterion, opt_func)

# model.learn(1)

In [1]:
def plot_accuracies(history):
    """ Plot the history of accuracies"""
    accuracies = [x['val_acc'] for x in history]
    plt.plot(accuracies, '-x')
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.title('Accuracy vs. No. of epochs')
    plt.show()



def plot_losses(history):
    """ Plot the losses in each epoch"""
    train_losses = [x.get('train_loss') for x in history]
    val_losses = [x['val_loss'] for x in history]
    plt.plot(train_losses, '-bx')
    plt.plot(val_losses, '-rx')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend(['Training', 'Validation'])
    plt.title('Loss vs. No. of epochs')
    plt.show()
    
plot_accuracies(history)

plot_losses(history)

NameError: name 'history' is not defined